
# **Name: Aditya Iche**
# **Roll No. CS6-27**         **Batch : C62**
# **PRN: 202401100136**
#Topic : Enron Email Dataset EDS Theory Assignment 1
Link to the dataset used: https://www.kaggle.com/datasets/wcukierski/enron-email-dataset

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("wcukierski/enron-email-dataset")

print("Path to dataset files:", path)

100%|██████████| 358M/358M [00:02<00:00, 143MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/wcukierski/enron-email-dataset/versions/2


## **Using a kaggle dataset for the Enrons. Following are the libraries used in the process of preprocessing the dataset and the doing the operations necessary for the assignment completion:**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import kagglehub


In [ ]:
df = pd.read_csv('/root/.cache/kagglehub/datasets/wcukierski/enron-email-dataset/versions/2/emails.csv')

# Preview the columns
df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [ ]:
#Extract Sender, Receiver, and Date
import re

# Define functions to extract metadata from message
def extract_sender(text):
    match = re.search(r"From: (.+)", str(text))
    return match.group(1).strip() if match else None

def extract_receiver(text):
    match = re.search(r"To: (.+)", str(text))
    return match.group(1).strip() if match else None

def extract_date(text):
    match = re.search(r"Date: (.+)", str(text))
    return match.group(1).strip() if match else None

# Apply to your DataFrame
df['sender'] = df['message'].apply(extract_sender)
df['receiver'] = df['message'].apply(extract_receiver)
df['date'] = df['message'].apply(extract_date)


In [ ]:
# Drop rows with missing fields
df.dropna(subset=['sender', 'receiver', 'date'], inplace=True)

# Convert date column to proper datetime
df['date'] = pd.to_datetime(df['date'], errors='coerce')

df = df.dropna(subset=['date'])

print(df['date'].dtype)


datetime64[ns, tzoffset('PDT', -25200)]


# **Now that the preprocessing is done, we can move on to the 20 problem statments that are relavant to this dataset and then their subsequent outputs:**

In [ ]:
# 1) Total number of emails in the dataset:
df.shape[0]


262812

In [ ]:
# 2. Number of unique senders:
df['sender'].nunique()


12036

In [ ]:
# 3. Top 5 most active senders:
df['sender'].value_counts().head(5)


,count
sender,
kay.mann@enron.com,8847
vince.kaminski@enron.com,6989
jeff.dasovich@enron.com,6525
pete.davis@enron.com,4997
tana.jones@enron.com,4936


In [ ]:
# 4. Number of emails sent by each sender:
emails_per_sender = df['sender'].value_counts()


In [ ]:
# 5. Top 5 most emailed recipients:
df['receiver'].value_counts().head(5)


,count
receiver,
pete.davis@enron.com,5006
tana.jones@enron.com,3812
sara.shackleton@enron.com,2911
jeff.dasovich@enron.com,2614
vkaminski@aol.com,2215


In [ ]:
# 6. Number of emails exchanged between specific sender-receiver pairs:
df.groupby(['sender', 'receiver']).size().sort_values(ascending=False).head(10)


sender                         receiver                                                
pete.davis@enron.com           pete.davis@enron.com                                        4995
vince.kaminski@enron.com       vkaminski@aol.com                                           1934
enron.announcements@enron.com  all.worldwide@enron.com                                     1199
                               all.houston@enron.com                                       1076
soblander@carrfut.com          soblander@carrfut.com                                        804
kay.mann@enron.com             suzanne.adams@enron.com                                      742
vince.kaminski@enron.com       shirley.crenshaw@enron.com                                   593
steven.kean@enron.com          maureen.mcvicker@enron.com                                   578
no.address@enron.com           All Enron Worldwide@ENRON                                    517
                               All Enron Worldwide@ENRON <??SAll Enron Worldwide@ENRON>     446
dtype: int64

In [ ]:
#7. Total number of emails per day:
df['date'].dt.date.value_counts().sort_index()


,count
date,
1986-05-01,1
1998-05-27,1
1998-09-01,1
1999-04-04,2
1999-04-05,9
...,...
2002-07-10,35
2002-07-11,34
2002-07-12,19


In [ ]:
# 8. Total number of emails sent each month:
df['month'] = df['date'].dt.to_period('M')
df.groupby('month').size()


<ipython-input-60-3e5cf4a2ebe9>:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['month'] = df['date'].dt.to_period('M')


,0
month,
1986-05,1
1998-05,1
1998-09,1
1999-04,74
1999-05,660
1999-06,645
1999-07,857
1999-08,1058
1999-09,1235


In [ ]:
#9. Day with the most emails sent:
df['date'].dt.date.value_counts().idxmax()


datetime.date(2001, 10, 25)

In [ ]:
#10. Number of emails sent during each hour of the day:
df['hour'] = df['date'].dt.hour
df['hour'].value_counts().sort_index()


,count
hour,
0,6337
1,13090
2,16859
3,17872
4,16989
5,14182
6,18896
7,21564
8,22387


In [ ]:
#11. Number of internal emails (sender and receiver domain is "enron.com"):
df['internal'] = df.apply(lambda x: 'enron.com' in x['sender'] and 'enron.com' in x['receiver'], axis=1)
df['internal'].sum()


np.int64(182747)

In [ ]:
#12. Number of external emails:
(~df['internal']).sum()


np.int64(80065)

In [ ]:
#13. Average number of characters in an email message:
df['msg_length'] = df['message'].apply(len)
df['msg_length'].mean()


np.float64(2631.1110870127695)

In [ ]:
#14. Maximum email length and who sent it:
df.loc[df['msg_length'].idxmax(), ['sender', 'msg_length']]


,114312
sender,no.address@enron.com
msg_length,1622422


In [ ]:
#15. Number of emails that mention “meeting” (case-insensitive):
df['message'].str.contains('meeting', case=False).sum()


np.int64(33190)

In [ ]:
#16. Number of emails sent on weekends:
df['weekday'] = df['date'].dt.dayofweek
df[df['weekday'] >= 5].shape[0]


9304

In [ ]:
#17. Correlation between email length and time of day:
df[['hour', 'msg_length']].corr()


,hour,msg_length
hour,1.000000,0.014173
msg_length,0.014173,1.000000


In [ ]:
#18. Number of duplicate messages:
df.duplicated(subset=['message']).sum()


np.int64(0)

In [ ]:
#19. Number of emails per sender per month:
monthly_counts = df.groupby([df['sender'], df['month']]).size().unstack(fill_value=0)
print(monthly_counts)

month                                  1986-05  1998-05  1998-09  1999-04  \
sender                                                                      
'todd'.delahoussaye@enron.com                0        0        0        0   
--migrated--bmishkin@ercot.com               0        0        0        0   
-nikole@excite.com                           0        0        0        0   
-persson@ricemail.ricefinancial.com          0        0        0        0   
01019@salespoint.dealerconnection.com        0        0        0        0   
...                                        ...      ...      ...      ...   
ztrizna@pol.net                              0        0        0        0   
zufferli@enron.com                           0        0        0        0   
zulie.flores@enron.com                       0        0        0        0   
zwharton@dawray.com                          0        0        0        0   
zzmacmac@aol.com                             0        0        0        0   

In [ ]:
# 20. Sender who sent emails on the most number of distinct days:
df['day'] = df['date'].dt.date
print(df.groupby('sender')['day'].nunique().sort_values(ascending=False).head(1))


sender
gerald.nemec@enron.com    362
Name: day, dtype: int64
